# AADT Training

## Imports

In [136]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError
from glob import glob


%matplotlib inline

## Global Variables

In [137]:
ROOT_DIR_PATH = os.path.abspath('..')

IMG_SIZE = 1024
VEHICLE_DETECTION_COUNT_PATH = os.path.join(ROOT_DIR_PATH, 'data/vehicle_counts_detection.csv')
AADT_PROCESSED_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/processed/')

NN_MODEL_PATH = os.path.join(ROOT_DIR_PATH, "models/nn_aadt_model.pth")

NORMALISED = True

## True Count Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Data Loading

In [138]:
pattern = os.path.join(AADT_PROCESSED_PATH, 'aadt_*.csv')

processed_aadt_file_paths = [os.path.join(AADT_PROCESSED_PATH, os.path.basename(x)) for x in glob(pattern)]
print("Processed aadt files: {}".format(processed_aadt_file_paths))

processed_aadt_df_list = []

for i in range(len(processed_aadt_file_paths)):
    processed_aadt_df = pd.read_csv(processed_aadt_file_paths[i])
    processed_aadt_df = processed_aadt_df.loc[:, ~processed_aadt_df.columns.str.contains('^Unnamed')]

    processed_aadt_df.name = processed_aadt_df.iloc[0]['Local Authority']+'_'+processed_aadt_df.iloc[0]['site_name'].replace('/', '_')

    processed_aadt_df_list.append(processed_aadt_df)

Processed aadt files: ['/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Hounslow_M4_2188A_2018.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Luton_M1_2557B_2018.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Hounslow_M4_2188B_2017.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Hounslow_M4_2188A_2017.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Hounslow_M4_2188B_2018.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Luton_M1_2557B_2017.csv', '/home/ah2719/FYP/Spatial_Finance_Transport/data/ground_truth_data/processed/aadt_Luton_M1_2557A_2018.csv']


## Custom Dataset

In [139]:
class CustomDataset(Dataset):
    def __init__(self, df, normalised=True):

        self.name = df.name
        
        self.labels = torch.tensor(df['all_motor_vehicles'].values.astype('float32'))
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.hour = torch.tensor(df['hour'].values.astype('float32')).unsqueeze(1)
        self.avg_mph = torch.tensor(df['avg_mph'].values.astype('float32')).unsqueeze(1)
        self.day = torch.tensor(df['day'].values.astype('float32')).unsqueeze(1)
        self.month = torch.tensor(df['month'].values.astype('float32')).unsqueeze(1)

        if normalised:
            vehicle_types = ['0-520cm_normalised', '521-660cm_normalised', '661-1160cm_normalised', '1160+cm_normalised', 'total_volume_normalised']
        else:
            vehicle_types = ['0-520cm', '521-660cm', '661-1160cm', '1160+cm', 'total_volume']

        self.small_vehicle = torch.tensor(df[vehicle_types[0]].values.astype('float32')).unsqueeze(1)
        self.mid_vehicle = torch.tensor(df[vehicle_types[1]].values.astype('float32')).unsqueeze(1)
        self.large_vehicle = torch.tensor(df[vehicle_types[2]].values.astype('float32')).unsqueeze(1)
        self.very_large_vehicle = torch.tensor(df[vehicle_types[3]].values.astype('float32')).unsqueeze(1)
        self.vehicle_count = torch.tensor(df[vehicle_types[4]].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.vehicle_count, self.small_vehicle, self.mid_vehicle, self.large_vehicle, self.very_large_vehicle, self.avg_mph, self.day, self.month, self.hour), dim=-1)
        self.y = self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.name, self.x[idx], self.y[idx]

In [140]:
dataset_list = []

for df in processed_aadt_df_list:

    custom_dataset = CustomDataset(df, normalised=NORMALISED)
    dataset_list.append(custom_dataset)

## Neural Network Models

In [141]:
class NeuralNetwork(nn.Module):
    def __init__(self, name):
        super(NeuralNetwork, self).__init__()

        self.name = name
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9, 9),
            nn.Linear(9,5),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(5,1),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [142]:
nn_model_list = []

for df in processed_aadt_df_list:
    nn_model = NeuralNetwork(name=df.name)
    nn_model_list.append(nn_model)

nn_model_list[0]

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=9, out_features=9, bias=True)
    (1): Linear(in_features=9, out_features=5, bias=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Linear(in_features=5, out_features=1, bias=True)
    (4): ReLU()
  )
)

## EarlyStopping

In [143]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

## Weights Initialisation

In [144]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

for nn_model in nn_model_list:
    nn_model.apply(init_weights)

## HyperParameters

In [145]:
learning_rate = 0.1
batch_size = 128
epochs = 5
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

MAPE = MeanAbsolutePercentageError()

## DataLoaders

In [146]:
train_dataloader_list = []
val_dataloader_list = []

for dataset in dataset_list:
    train_split = 0.8
    train_data, val_data = random_split(dataset, [train_split, 1-train_split])

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)
    
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=2,
                    persistent_workers=False)
    
    train_dataloader_list.append(train_dataloader)
    val_dataloader_list.append(val_dataloader)

## Training & Validation

In [147]:
def run_epoch(ep_id, action, loader, model, optimizer, criterion, early_stopper):
    losses = [] # Keep list of accuracies to track progress
    is_training = action == "train" # True when action == "train", else False 

    # Looping over all batches
    for batch_idx, batch in enumerate(loader): 
        dl_name, x, y = batch

        # Assert we are training the correct model with the correct dataset!
        assert dl_name[0] == model.name

        # Resetting the optimizer gradients
        optimizer.zero_grad()

        # Setting model to train or test
        with torch.set_grad_enabled(is_training):
            
            # Feed batch to model
            logits = model(x).squeeze(1)

            # Calculate the loss based on predictions and real labels
            loss = criterion(logits, y)
            mape_loss = MAPE(logits, y)

            # If training, perform backprop and update weights
            if is_training:
                loss.backward()
                optimizer.step()

            # Append current batch accuracy
            losses.append(mape_loss.detach().numpy())

            # Print some stats every 50th batch 
            if batch_idx % 50 == 0:
                print(f"{action.capitalize()}, Epoch: {ep_id+1}, Batch {batch_idx}: Loss = {loss.item()}")

        if not is_training:
            if early_stopper.early_stop(mape_loss.detach().numpy()):
                print("Entered Early Stopping")             
                break
                    
    # Return accuracies to main loop                 
    return losses

In [148]:
def train(epochs, train_dl, val_dl, model, optimizer, criterion, early_stopper):

    # Keep lists of accuracies to track performance on train and test sets
    train_losses = []
    val_losses = []

    # Looping over epochs
    for epoch in range(epochs):
        
        # Looping over train set and training
        train_loss = run_epoch(epoch, "train", train_dl, model, optimizer, criterion, early_stopper=early_stopper)

        # Looping over test set
        val_loss = run_epoch(epoch, "val", val_dl, model, optimizer, criterion, early_stopper=early_stopper) 

        # Collecting stats
        train_losses += train_loss
        val_losses += val_loss         
            
    return train_losses, val_losses

In [149]:
def main(epochs, train_dl_list, val_dl_list, model_list):
    all_train_losses = []
    all_val_losses = []

    for (model, train_dl, val_dl) in zip(model_list, train_dl_list, val_dl_list):

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        loss_fn = nn.MSELoss()

        early_stopper = EarlyStopper(patience=3, min_delta=10)
        
        train_losses, val_losses = train(epochs, train_dl, val_dl, model, optimizer=optimizer, criterion=loss_fn, early_stopper=early_stopper)

        all_train_losses.append((model.name, train_losses))
        all_val_losses.append((model.name, val_losses))

    return all_train_losses, all_val_losses

In [150]:
all_train_losses, all_val_losses = main(epochs=epochs, train_dl_list=train_dataloader_list, val_dl_list=val_dataloader_list, model_list=nn_model_list)

Train, Epoch: 1, Batch 0: Loss = 2605707264.0
Val, Epoch: 1, Batch 0: Loss = 2566708480.0
Train, Epoch: 2, Batch 0: Loss = 2567570944.0
Val, Epoch: 2, Batch 0: Loss = 2380093440.0
Train, Epoch: 3, Batch 0: Loss = 2385408256.0
Val, Epoch: 3, Batch 0: Loss = 1887774208.0
Train, Epoch: 4, Batch 0: Loss = 1904391808.0
Val, Epoch: 4, Batch 0: Loss = 1005757184.0
Train, Epoch: 5, Batch 0: Loss = 1038158336.0
Val, Epoch: 5, Batch 0: Loss = 125666496.0
Train, Epoch: 6, Batch 0: Loss = 152278592.0
Val, Epoch: 6, Batch 0: Loss = 305996128.0
Train, Epoch: 7, Batch 0: Loss = 271082144.0
Val, Epoch: 7, Batch 0: Loss = 187022448.0
Train, Epoch: 8, Batch 0: Loss = 166289120.0
Val, Epoch: 8, Batch 0: Loss = 47824860.0
Train, Epoch: 9, Batch 0: Loss = 62284528.0
Val, Epoch: 9, Batch 0: Loss = 104758336.0
Train, Epoch: 10, Batch 0: Loss = 126694136.0
Val, Epoch: 10, Batch 0: Loss = 59348840.0
Train, Epoch: 1, Batch 0: Loss = 4948804608.0
Val, Epoch: 1, Batch 0: Loss = 4884428288.0
Train, Epoch: 2, Batch

### Loss Curves Plot

In [151]:
for model_train_losses in all_train_losses:
    (name, train_losses) = model_train_losses
    
    fig = px.line(train_losses, title=name)
    fig.show()

In [152]:
for model_val_losses in all_val_losses:
    (name, val_losses) = model_val_losses
    
    fig = px.line(val_losses, title=name)
    fig.show()

### Random val samples

In [153]:
for nn_model, val_data in zip(nn_model_list, dataset_list):    
    
    preds = []
    ground_truth = []
    vehicle_counts = []


    for i in range(10):
        vehicle_count = 0
        random_idx = np.random.randint(0,len(val_data))
        dataset_name, x, y = val_data[random_idx]

        assert dataset_name == nn_model.name

        vehicle_count = x[0]
        vehicle_counts.append(vehicle_count)
        ground_truth.append(float(y))
        pred_y = float(nn_model(x)[0])
        preds.append(pred_y)

    df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})

    fig = px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'], title=nn_model.name)

    fig.show()

## Save Model

In [154]:
torch.save(nn_model.state_dict(), NN_MODEL_PATH)